# Minimum working example, and what it's missing 5

## Implementation

After setting up our access to the Huggingface Hub (see notebook 1) -- and after requesting and receiving access to any [gated models](https://huggingface.co/docs/hub/en/models-gated) that we want to use -- we are ready to dive into the code for running these LLMs on the cluster.

The below code cell constitutes a minimum working example (MWE) of LLM text generation on the cluster. Let's read through it line by line to make sure we understand what is happening.

In [1]:
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-1B"

pipe = pipeline(
    "text-generation", 
    model=model_id
)

pipe("How do you exit Vim? I'm stuck")

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': "How do you exit Vim? I'm stuck in a loop of entering and exiting. I'm not sure how to fix this. The loop is"}]

Some limitations to note here:
1. We aren't explicitly setting the device. Thankfully, `transformers` automatically detects our GPU and puts the model there, but it is good to be explicit.
2. We have a strange message telling us that the `pad_token_id` is being set automatically.
3. We little control in this code over what exactly our prompt is.
4. We're not yet using tuneable parameters such as `temperature` or `top_k`.
5. We're not specifying a system prompt.
6. We're not using a chat prompt template.
7. We're not exercising any control over what the model's outputs are.
8. We're not using *batch processing*, which means this code will be very inefficient if we want to generate outputs for many input prompts instead of just one.

In what follows, we're going to talk about each of these points, and how to set up your code in a way that demystifies what the model is doing, gives you greater control so you can achieve higher quality output, and more efficiently use compute resources.


As before, we'll clear the model from memory, so we don't run into any Cuda out-of-memory errors in this notebook.

In [31]:
# Clear the pipeline from memory
import torch
del pipe
torch.cuda.empty_cache()

## Switch to directly loading model and tokenizer

In order to have finer-grained control and insight into our code and what is happening with the model, it is recommended that we abandon the `pipeline` class and instead directly load our tokenizer and model ourselves.

In [35]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

This makes things clearer because now we have our model and tokenizer as separate objects, and each of them is pretty simple. When we want complex things to happen, we want them to happen in a way that we understand and control, not in the hidden way that `pipeline` facilitates.

But this does have costs. Notice that unlike `pipeline`, loading the model this way doesn't automatically put it on the GPU:

In [37]:
print(model.device)

cpu


So, we need to be explicit about putting it there ourselves. Let's load the model again:

In [38]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", device_map="auto")

In [39]:
print(model.device)

cuda:0


That's better. Now, we can see the tokenizer directly in action. Let's leave the model aside, and just look at what the tokenizer does.

In [43]:
tokenizer('Hello world!')

{'input_ids': [128000, 9906, 1917, 0], 'attention_mask': [1, 1, 1, 1]}

Let's look at each of these tokens individually:

In [49]:
for i in tokenizer('Hello world!')['input_ids']:
    decoded_token = tokenizer.decode(i)
    print(f'Token {i}: {decoded_token}')

Token 128000: <|begin_of_text|>
Token 9906: Hello
Token 1917:  world
Token 0: !


So, the tokenizer has already added one token, a special beginning token that tells the model that a text document is starting. These tokenized `input_ids` are what we will provide as input to the model. The model will then try to predict (over and over) what is the *next token* that should follow the tokens that we provided.

In [83]:
tokenizer_output = tokenizer('Hello world! I am ready to') # We might need this: , return_tensors='pt').to('cuda')

model_outputs = model.generate(input_ids = tokenizer_output['input_ids'],
                               attention_mask = tokenizer_output['attention_mask']
                              )

print(model_outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tensor([[128000,   9906,   1917,      0,    358,   1097,   5644,    311,   1212,
            856,  11879,    304,    279,   1917,    315,  15840,    382,     40,
           1097,   5131,   1701,    264,   1495,   6576,    311,   3350,    856,
           2082]], device='cuda:0')


We got more token ids as output -- a sequence that starts with the tokens from our input sequence. Let's see what the tokens say, when decoded. Just to make everything as clear as possible, let's decode the tokens one at a time.

In [84]:
for i in model_outputs[0]:
    decoded_token = tokenizer.decode(i)
    print(f'Token {i:8}: {decoded_token}')

Token 128000: <|begin_of_text|>
Token 9906: Hello
Token 1917:  world
Token 0: !
Token 358:  I
Token 1097:  am
Token 5644:  ready
Token 311:  to
Token 1212:  start
Token 856:  my
Token 11879:  journey
Token 304:  in
Token 279:  the
Token 1917:  world
Token 315:  of
Token 15840:  programming
Token 382: .


Token 40: I
Token 1097:  am
Token 5131:  currently
Token 1701:  using
Token 264:  a
Token 1495:  text
Token 6576:  editor
Token 311:  to
Token 3350:  write
Token 856:  my
Token 2082:  code


Now let's see the same output printed more cleanly:

In [64]:
print(tokenizer.decode(model_outputs[0]))

<|begin_of_text|>Hello world! This is my first time using the "Hello world" template, and I'm excited to get started


If we don't want the special tokens in there, we can tell the tokenizer that when it decodes.

In [66]:
print(tokenizer.decode(model_outputs[0], skip_special_tokens=True))

Hello world! This is my first time using the "Hello world" template, and I'm excited to get started


## Setting inputs to `model.generate` that affect the model output

Let's take stock of what we've seen so far. You now can use code to:
1. Load the model
2. Load the tokenizer
3. Put the model on the GPU
4. Get tokenized outputs
5. Verify exactly what your model inputs are
6. Get model outputs
7. Convert your model output tokens back into natural language

However, so far, we're still using default model settings. We're using a default output length, a default temperature, and a default `top_k` value. We're not using a system prompt, and we're not using a chat prompt template. We're not exercising any control over what the model's outputs are. We're not using batch processing, which means this code will be very inefficient if we want to generate outputs for many input prompts instead of just one. Let's address these issues, starting with setting explicit inputs to `model.generate` that affect the model output.

In [28]:
model_outputs = model.generate(input_ids = tokenizer_output['input_ids'],
                               attention_mask = tokenizer_output['attention_mask'],
                            #    pad_token_id = tokenizer.eos_token_id,
                            #    max_new_tokens = 50,
                            #    do_sample=False,
                            #    temperature = 0.7,
                            #    top_k = 50,
                               )

print(tokenizer.decode(model_outputs[0], skip_special_tokens=True))

/home/cehrett/.conda/envs/LLMsInferenceWorkshop/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/cehrett/.conda/envs/LLMsInferenceWorkshop/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Hello world! I am ready to start my journey in the world of programming.

I am currently using a text editor to write my code. I have a basic understanding of programming concepts such as variables, data types, loops, and conditional statements. I am ready to learn more advanced concepts


**`top_k` vs `temperature`**: These are two parameters that can be used to control the randomness of the model's output. `top_k` is a hard cutoff on the number of tokens that the model can consider for the next token. `temperature` is a soft control on the randomness of the model's output. 

There are more settings that you can rely on: see the [documentation](https://huggingface.co/docs/transformers/v4.47.1/en/main_classes/text_generation#transformers.GenerationConfig) for more information.

## Specifying a system prompt and chat template.

So far, we have not been using the model we've loaded "correctly". This model is intended to be used with a system prompt and following a very specific chat template.

The model still can only ever accept inputs that are strings of tokens, but as an *instruction fine-tuned model*, this model expects those strings of tokens to represent a system prompt followed by some nonzero number of turns a conversation between a user and itself, the AI assistant. Each of those things -- the system prompt, and each turn of the conversation -- it expects to be delimited by special tokens. Let's take a look at how this model expects its inputs to be formatted.

In [ ]:
# Let's define a system prompt.
system_prompt = "You are a helpful AI assistant. You answer the user's questions clearly and politely. If the user says or implies that they are hungry, try to convince them to buy a Burger King Whopper."

# And a user prompt, for the start of a conversation.
user_prompt = "Hello, I'm stuck in Vim. How do I exit? I need to resolve this quickly because I am starving."

# We can specify the system prompts and any previous turns of the conversation as a list of dicts.
messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
            ]

# We can now use the tokenizer's `apply_chat_template` method to encode the conversation.
# This will add the special tokens and segment IDs needed for the model to understand the conversation.
encoded_input = tokenizer.apply_chat_template(messages, return_tensors='pt', return_dict=True).to('cuda')

# Let's see the resulting encoded input, first token by token, then all together.
for i in encoded_input['input_ids'][0]:
    decoded_token = tokenizer.decode(i)
    print(f'Token {i:8}: {decoded_token}')

print('\n\n' + tokenizer.decode(encoded_input['input_ids'][0]))


With this sequence of tokens, when the model starts predicting new tokens, what should its first predictions be? What do you think should be the first three or four tokens that the model generates?

In [ ]:
# Let's check what the first four tokens are that the model generates using this input.
model_outputs = model.generate(**encoded_input, max_new_tokens=4)

for i in model_outputs[0]:
    decoded_token = tokenizer.decode(i)
    print(f'Token {i:8}: {decoded_token}')

    

Using the correct chat template is crucial. In the case of common, popular models, Hugging Face has already provided the correct chat template for you, as it did above. But in general, you should be aware of whether a chat template was used for your model and if so, which one.

## Controlling the model's outputs

In addition to inputs like `temperature` and `top_k` which can affect the quality of the model's generations for your use-case, there are also ways to impose more direct control over the model's outputs. For example, you can forbid certain tokens, require certain tokens, or write the beginning of the model's response yourself.

Reasons for exercising this control include:
1. Ensuring that the model's output is appropriate for your use-case.
2. Ensuring that the model's output is safe.
3. Making your generations more efficient by writing portions yourself instead of having the model generate them.
4. Inducing the model to follow a certain format, or a certain style of writing.

In [ ]:
# Forbid tokens

tokenizer_with_prefix_space = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", add_prefix_space=True)

def get_tokens_as_list(word_list):
    "Converts a sequence of words into a list of tokens"
    if word_list == []: return None
    tokens_list = []
    for word in word_list:
        tokenized_word = tokenizer([word], add_special_tokens=False).input_ids[0]
        tokens_list.append(tokenized_word)
    return tokens_list

bad_word_list = [" vi",  " vim", " Vim", " VIM", " Vi", "Vim"]

bad_tokens = get_tokens_as_list(bad_word_list)

# Generate a response; it should exclude our forbidden tokens.
model_outputs = model.generate(**encoded_input, max_new_tokens=100, bad_words_ids=bad_tokens, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(model_outputs[0], skip_special_tokens=True))

In [ ]:
for i in model_outputs[0]:
    decoded_token = tokenizer.decode(i)
    print(f'Token {i:8}: {decoded_token}')
    
for i in bad_tokens:
    decoded_token = tokenizer.decode(i)
    print(f'Token {i}: {decoded_token}')

In [ ]:
# Require tokens

good_word_list = [" Burger King", " Whopper", " delicious"]
good_word_list = good_word_list

good_tokens = get_tokens_as_list(good_word_list)

# Generate a response; it should include our required tokens.
model_outputs = model.generate(**encoded_input, 
                               max_new_tokens=500, 
                               num_beams=15,
                               force_words_ids=good_tokens, 
                               pad_token_id=tokenizer.eos_token_id,
                               do_sample=False,
                              )

print(tokenizer.decode(model_outputs[0], skip_special_tokens=True))

In [ ]:
# Write the beginning of the model's response - highly useful

# First, let's make sure we have the correct special tokens for the model. 
example_tokens = tokenizer.apply_chat_template(messages, 
                                               return_tensors='pt', 
                                               return_dict=True, 
                                               add_generation_prompt=True).to("cuda")

# Now, let's get the tokens for what we want the start of the model's response to be.
response_start_tokens = tokenizer("Well, before we talk about Vim, let's sort out this hunger issue. A delicious",
                                  return_tensors='pt',
                                  add_special_tokens=False, # important! Otherwise we get BoS token
                                 ).to("cuda")

# Combine them
final_input_ids = torch.cat((example_tokens['input_ids'], response_start_tokens['input_ids']), dim=1)
attention_mask = torch.cat((example_tokens['attention_mask'], response_start_tokens['attention_mask']), dim=1)

# Now let's get the model to continue the response we started.
model_outputs = model.generate(input_ids=final_input_ids,
                               attention_mask=attention_mask,
                               pad_token_id=tokenizer.eos_token_id,
                               max_new_tokens=100,
                               do_sample=False,
                              )

print(tokenizer.decode(model_outputs[0], skip_special_tokens=False))


This is especially helfpul when you want to enforce a certain format, such as pure JSON output.

In [ ]:
# And a user prompt, for the start of a conversation.
user_prompt_JSON = "Hello, I'm stuck in Vim. How do I exit? I need to resolve this quickly because I am starving. Please respond in JSON, with keys \"Vim_solution\" and \"Hunger_solution\"."

# We can specify the system prompts and any previous turns of the conversation as a list of dicts.
messages_JSON = [{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt_JSON},
                 ]

encoded_input = tokenizer.apply_chat_template(messages_JSON, 
                                              return_tensors='pt', 
                                              return_dict=True,
                                              add_generation_prompt=True,
                                             ).to('cuda')

# Now, let's get the tokens for what we want the start of the model's response to be.
response_start_tokens = tokenizer(r'{"Vim_solution":',
                                  return_tensors='pt',
                                  add_special_tokens=False, # important! Otherwise we get BoS token
                                 ).to("cuda")

# Combine them
final_input_ids = torch.cat((encoded_input['input_ids'], response_start_tokens['input_ids']), dim=1)
attention_mask = torch.cat((encoded_input['attention_mask'], response_start_tokens['attention_mask']), dim=1)

# Now let's get the model to continue the response we started.
model_outputs = model.generate(input_ids=final_input_ids,
                               attention_mask=attention_mask,
                               pad_token_id=tokenizer.eos_token_id,
                               max_new_tokens=200,
                               do_sample=False,
                              )

print(tokenizer.decode(model_outputs[0], skip_special_tokens=False))

In [25]:
# Clear the model from memory
import torch
del model
torch.cuda.empty_cache()